# 4.- Training Configuration

In [20]:
import os
import requests
import tarfile
import re
from git import Repo

In [21]:
def download_file(url, destination_folder, file_name):

    os.makedirs(destination_folder, exist_ok=True)
    path_to_write = f"{destination_folder}/{file_name}"
    response = requests.get(url)

    if response.status_code == 200:
        with open(path_to_write, 'wb') as file:
            file.write(response.content)
        print(f"File downloaded and saved to: {path_to_write}")
    else:
        print(f"Failed to download the file, status code: {response.status_code}")


In [22]:
chosen_model = 'ssd-mobilenet-v2-fpnlite-320'

MODELS_CONFIG = {
    'ssd-mobilenet-v2': {
        'model_name': 'ssd_mobilenet_v2_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz',
    },
    'efficientdet-d0': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
    },
    'ssd-mobilenet-v2-fpnlite-320': {
        'model_name': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz',
    },
}

model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']

## Download Model

In [25]:
chosen_model = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint
destination_folder = "../external/train_models"
model_file_name = f"{chosen_model}.tar.gz"

In [ ]:
download_file(download_tar, destination_folder, model_file_name)

In [31]:
tar = tarfile.open(f"{destination_folder}/{model_file_name}")
tar.extractall(path = destination_folder)
tar.close()

## Download Model Config File

In [ ]:
url_config = f"https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/{model_name}.config"
config_file_name = f"{model_name}.config"

In [ ]:
download_file(url_config, destination_folder, config_file_name)

## Clone Github Tensorflow Models Garden Repository

In [15]:
def clone_repository(repo_url, local_dir):
    # Create the local directory if it does not exist
    if not os.path.exists(local_dir):
        os.makedirs(local_dir)
        print(f"Directory '{local_dir}' created.")
    else:
        print(f"Directory '{local_dir}' already exists.")
    
    # Clone the repository into the specified directory
    try:
        print(f"Cloning {repo_url} into {local_dir}")
        Repo.clone_from(repo_url, local_dir)
        print("Repository successfully cloned.")
    except Exception as e:
        print(f"Failed to clone the repository: {e}")


In [16]:
# URL of the GitHub repository you want to clone
repo_url = 'https://github.com/tensorflow/models.git'

# Local path where you want to save the cloned repository
train_code = '../src/'

# Execute the function
clone_repository(repo_url, train_code)

Directory '../src/' created.
Cloning https://github.com/tensorflow/models.git into ../src/
Repository successfully cloned.


## Define Parameters

In [ ]:
num_steps = 40000
batch_size = 2

In [27]:
# Set file locations and get number of classes for config file
pipeline_fname = destination_folder + '/' + base_pipeline_file
fine_tune_checkpoint = destination_folder + '/' + model_name + '/checkpoint/ckpt-0'
num_classes = 1

train_record_fname = '/content/train.tfrecord'
val_record_fname = '/content/val.tfrecord'
label_map_pbtxt_fname = '/content/labelmap.pbtxt'
model_dir = '/content/training/'

## Create Config File

In [30]:
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open(f"{train_code}/custom_pipeline_file.config", 'w') as f:

    # Set fine_tune_checkpoint path
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)

    # Set tfrecord files for train and test datasets
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(val_record_fname), s)

    # Set label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set batch_size
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)

    # Set number of classes num_classes
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)

    # Change fine-tune checkpoint type from "classification" to "detection"
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)

    # If using ssd-mobilenet-v2, reduce learning rate (because it's too high in the default config file)
    if chosen_model == 'ssd-mobilenet-v2':
      s = re.sub('learning_rate_base: .8',
                 'learning_rate_base: .08', s)

      s = re.sub('warmup_learning_rate: 0.13333',
                 'warmup_learning_rate: .026666', s)

    # If using efficientdet-d0, use fixed_shape_resizer instead of keep_aspect_ratio_resizer (because it isn't supported by TFLite)
    if chosen_model == 'efficientdet-d0':
      s = re.sub('keep_aspect_ratio_resizer', 'fixed_shape_resizer', s)
      s = re.sub('pad_to_max_dimension: true', '', s)
      s = re.sub('min_dimension', 'height', s)
      s = re.sub('max_dimension', 'width', s)

    f.write(s)


writing custom configuration file
